In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os

from helper_functions import elementlines, nearestlines, theoretical_energy

In [ ]:
%matplotlib qt

plt.rcParams['font.size'] = '16'
plt.rcParams["figure.figsize"] = (15,8)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'monospace'

In [ ]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'


In [ ]:
# put all files in a list
files = os.listdir(path)

# Quantification with AZtec

In [ ]:
aztec_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec k-factors and quantification.xlsx'
nans = np.array([0,3,6,9,10,11,12,15,16,19,22,25,28,31,34,37,40,43,46,49,52,55,56,57,60])+1
df = pd.read_excel(aztec_file, sheet_name=4, skiprows=nans)

In [ ]:
df['Name'] = df['File']
df['Name'] = df['Name'].str.replace('_', ' ')
df['Name'] = df['Name'].str.replace('GaAs', '(a)')
df['Name'] = df['Name'].str.replace('GaSb', '(b)')
df['Name'] = df['Name'].str.replace('processTime', 'PT')
df['Name'] = df['Name'].str.replace('scratched', 'scr')
df['Line'] = df['Element'] + '_' + df['Line series'] + 'a'
df['Line'] = df['Line'].str.replace(' ', '')
df.head(3)

In [ ]:
# make df2 as a deep copy of df
df2 = df.copy()
# drop all columns except Name, Line, Int. correction, k-ratio, %at
df2 = df2[['Name', 'Line', 'Int. correction', 'k-ratio', '%at']]
# rename columns
df2['i'] = 0.0
df2.columns = ['Name', 'Line', 'corr', 'k', 'AZ at%', 'i']
df2.head(3)

In [ ]:
# plot '%at' col with "Line" as color
fig = px.scatter(df, x='Name', y='%at', color='Element', hover_name='Name', hover_data=['%at', 'Element', 'Line series', 'File'])
fig.update_xaxes(tickangle=90)
fig.update_layout(title='AZtec quantification  <br>(a)=GaAs, (b)=GaSb, scr=scratched')
fig

In [ ]:
df['File'].unique()

In [ ]:
df['File'].unique()[0]

In [ ]:
s = hs.load(path + df['File'].unique()[0] + '.emsa', signal_type='EDS_SEM')
s.metadata

In [ ]:
def calculate_composition(file_number, df, df2):
    file = df['File'].unique()[file_number]
    df_line = df[df['File']==file]

    elements = df_line['Element'].unique()
    intCorr = df_line['Int. correction'].unique()
    lines = df_line['Line'].unique()
    kRatio = df_line['k-ratio'].unique()
    atAZtec = df_line['%at'].unique()

    s = hs.load(path + file + '.emsa', signal_type='EDS_SEM')
    s = s.isig[0.4:] # 0.4 keV

    # if Vacc is lower than 20, slice at Vacc
    if s.metadata.Acquisition_instrument.SEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.SEM.beam_energy]

    s.add_elements(elements)
    s.add_lines()

    m = s.create_model()
    m.fit()
    m.fit_background()

    intensities_raw_modelled = [m[lines[0]].A.value, m[lines[1]].A.value]

    # add intensities_raw_modelled to df
    
    conditions = [
        (df['File'] == file) & (df['Line'] == lines[0]),
        (df['File'] == file) & (df['Line'] == lines[1])
    ]
    df['i'] = np.select(conditions, intensities_raw_modelled, default=np.nan)


    # df2['i'] = np.select(conditions, intensities_raw_modelled)





    # intensities_corrected = [i/k for i,k in zip(intensities_raw_modelled, kRatio)]
    i_k = [i/k for i,k in zip(intensities_raw_modelled, kRatio)]
    ic_k = [i*c/k for i,c,k in zip(intensities_raw_modelled, intCorr, kRatio)]
    i_c_k = [i/c/k for i,c,k in zip(intensities_raw_modelled, intCorr, kRatio)]
    # intensities_corrected = [i/c/k for i,c,k in zip(intensities_raw_modelled, intCorr, kRatio)]

    p = file[:15] + '\t i/k' + '\t i*c/k' + '\t i/c/k' + '\t AZtec'
    print(p)
    # print(lines[0], '\t\t', round(i_k[0],2), '\t', round(ic_k[0],2), '\t', round(i_c_k[0],2), '\t', round(atAZtec[0],2))
    print(lines[0], '\t\t', round(i_k[0]/sum(i_k),2), '\t', round(ic_k[0]/sum(ic_k),2), '\t', round(i_c_k[0]/sum(i_c_k),2), '\t', round(atAZtec[0]/sum(atAZtec),2))
    print(lines[1], '\t\t', round(i_k[1]/sum(i_k),2), '\t', round(ic_k[1]/sum(ic_k),2), '\t', round(i_c_k[1]/sum(i_c_k),2), '\t', round(atAZtec[1]/sum(atAZtec),2))



    # concentrations = [round(intensities_corrected[i]/sum(intensities_corrected), 4) for i in [0,1]]
    # print(concentrations, '\n', atAZtec)

    
    return intensities_raw_modelled

In [ ]:
calculate_composition(1, df, df2)

In [ ]:
df2.head(6)

In [ ]:
intensities_list = []
for i in range(len(df['File'].unique())-2): # -2 because of the two maps
    # print(df['File'].unique()[i])
    i_list = calculate_composition(i, df, df2)
    intensities_list.append(i_list[0])
    intensities_list.append(i_list[1])


In [ ]:
len(intensities_list)
# len(df2['i'])

In [ ]:
intensities_list += [np.nan]*4

In [ ]:
df2['i'] = intensities_list
df['i'] = intensities_list

df2.head(3)

In [ ]:
fig = px.scatter(df, x='Name', y='i', color='Line', hover_name='Name', hover_data=['%at', 'Element', 'Line series', 'File'])

fig.update_xaxes(tickangle=90)
fig.update_yaxes(range=[0,400e3])

fig.update_layout(title='Counts in the peaks  <br>(a)=GaAs, (b)=GaSb, scr=scratched')

fig.write_html('i.html')

fig

In [ ]:
df2['i/k'] = df2['i']/df2['k']
df2['i*c/k'] = df2['i']*df2['corr']/df2['k']
df2['i/c/k'] = df2['i']/df2['corr']/df2['k']


In [ ]:
# go through unique names and calculate i/k[0]/sum(i/k) and i/k[1]/sum(i/k)

for name in df2['Name'].unique():
    df2['i/k'][df2['Name']==name] = df2['i/k'][df2['Name']==name]/sum(df2['i/k'][df2['Name']==name])
    df2['i*c/k'][df2['Name']==name] = df2['i*c/k'][df2['Name']==name]/sum(df2['i*c/k'][df2['Name']==name])
    df2['i/c/k'][df2['Name']==name] = df2['i/c/k'][df2['Name']==name]/sum(df2['i/c/k'][df2['Name']==name])

In [ ]:
fig = px.scatter(df2.sort_values(by=["Line"]), x='Name', y='AZ at%', color='Line', hover_name='Name', hover_data=['AZ at%', 'i', 'i/k', 'i*c/k', 'i/c/k'])

fig.update_traces(marker=dict(size=10,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))


fig.update_xaxes(tickangle=90)
fig.update_layout(title='AZ at%')
fig.write_html('AZ_comp.html')

fig

In [ ]:
c0 = 2
c2 = 6
co = np.array([px.colors.sequential.Blues[c0:c2],
px.colors.sequential.Teal[c0:c2],
px.colors.sequential.Peach[c0:c2],
px.colors.sequential.Burg[c0:c2],
px.colors.sequential.Greys[c0:c2]])

In [ ]:
fig = go.Figure()

lines_list = ['As_Ka', 'As_La', 'Ga_Ka', 'Ga_La', 'Sb_La']

for l in lines_list:
    # df2[df2['Line'] =='As_Ka']
        # print(df2[df2['Line'] =='As_Ka'][key])
    i = lines_list.index(l)
    mode = 'markers'
    bol = df2['Line'] == l
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i/k'], mode=mode, name=l, marker=dict(size=10, color=co.T[0][i]), legendgroup=l)
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i*c/k'], mode=mode, name=l, marker=dict(size=10, color=co.T[1][i]), legendgroup=l)
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i/c/k'], mode=mode, name=l, marker=dict(size=10, color=co.T[2][i]), legendgroup=l)


    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i']/df2[bol]['i'].mean(), mode=mode, name=l, marker=dict(size=10, color=co.T[2][i]), legendgroup=l)
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['AZ at%']/100, mode='markers', name='AZ', marker=dict(size=5, color=co.T[3][i]))

    fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i'], mode=mode, name=f'I({l})', marker=dict(size=10, color=co.T[2][i], symbol='square'))
    fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['AZ at%']/100*df2[bol]['i'].mean(), mode='markers', name='AZ at% * I(mean)', marker=dict(size=5, color=co.T[3][i]))

fig.update_yaxes(type='log', title='Counts and [AZ at% * I(mean)]')
fig.add_hline(y=0.5*df2[bol]['i'].mean(), line_width=1, line_dash="dash", line_color="black")
fig.update_layout(title='Intensity (squares) and AZ at% (circles)')

fig


In [ ]:
fig = go.Figure()
lines_list = ['Sb_La']
for l in lines_list:
    i = lines_list.index(l)
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i/k'], mode='markers+lines', name='i/k', marker=dict(size=5, color=co.T[0][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i*c/k'], mode='markers+lines', name='i*c/k', marker=dict(size=5, color=co.T[1][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i/c/k'], mode='markers+lines', name='i/c/k', marker=dict(size=5, color=co.T[2][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['AZ at%']/100, mode='markers+lines', name='AZ at%', marker=dict(size=5, color=co.T[3][i]))
fig.update_layout(title='Sb La')
fig


In [ ]:
fig = go.Figure()
lines_list = ['Ga_La']
for l in lines_list:
    i = lines_list.index(l)
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i/k'], mode='markers+lines', name='i/k', marker=dict(size=5, color=co.T[0][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i*c/k'], mode='markers+lines', name='i*c/k', marker=dict(size=5, color=co.T[1][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['i/c/k'], mode='markers+lines', name='i/c/k', marker=dict(size=5, color=co.T[2][i]))
    fig.add_scatter(x=df2[df2['Line'] == l]['Name'], y = df2[df2['Line'] == l]['AZ at%']/100, mode='markers+lines', name='AZ at%', marker=dict(size=5, color=co.T[3][i]))
fig.update_layout(title=lines_list[0])
fig


In [ ]:
fig = go.Figure()
lines_list = ['Sb_La']
for l in lines_list:
    i = lines_list.index(l)
    bol = df2['Line'] == l
    fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i']/df2[bol]['i'].max(), mode='markers+lines', name='i', marker=dict(size=5, color=co.T[0][i]))
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i*c/k'], mode='markers+lines', name='i*c/k', marker=dict(size=5, color=co.T[1][i]))
    # fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['i/c/k'], mode='markers+lines', name='i/c/k', marker=dict(size=5, color=co.T[2][i]))
    fig.add_scatter(x=df2[bol]['Name'], y = df2[bol]['AZ at%']/100, mode='markers+lines', name='AZ at%', marker=dict(size=5, color=co.T[3][i]))
fig.update_layout(title=lines_list[0])
fig


In [ ]:
fig = go.Figure()

fig.add_scatter(x=df)

In [ ]:
fig = px.scatter(df2, x='Name', y='i/k', color='Line', hover_name='Name', hover_data=['AZ at%', 'i', 'i/k', 'i*c/k', 'i/c/k'])


fig.update_xaxes(tickangle=90)
fig.update_layout(title='i/k')
fig.write_html('i_k.html')

fig

In [ ]:
fig = px.scatter(df2, x='Name', y='i*c/k', color='Line', hover_name='Name', hover_data=['AZ at%', 'i', 'i/k', 'i*c/k', 'i/c/k'])
fig.update_xaxes(tickangle=90)
fig.update_layout(title='i*c/k')
fig.write_html('ic_k.html')

fig

In [ ]:
fig = px.scatter(df2, x='Name', y='i/c/k', color='Line', hover_name='Name', hover_data=['AZ at%', 'i', 'i/k', 'i*c/k', 'i/c/k'])
fig.update_xaxes(tickangle=90)
fig.update_layout(title='i/c/k')
fig.write_html('i_c_k.html')
fig

In [ ]:
i1 = int(m['As_Ka'].A.value) * 1.03
k1 = 0.01876
i2 = int(m['Ga_Ka'].A.value) * 1.0
k2 = 0.03083

In [ ]:
print('As Ka raw I: ', i1, '     As Ka corrected: ', i1 / k1)
print('Ga Ka raw I: ', i2, '     Ga Ka corrected: ', i2 / k2)

t1 = i1 / k1
t2 = i2 / k2
print('As:' , t1 / (t1 + t2))
print('Ga:' , t2 / (t1 + t2))

# without AZtec intensity correction
# As: 0.4943
# Ga: 0.5057

In [ ]:
s = hs.load(path + files[3], signal_type='EDS_TEM')
s = s.isig[0.4:] # 0.4 keV

s.add_elements(['As','Ga'])
s.add_lines()

m = s.create_model()
m.fit()
m.fit_background()

In [ ]:
i1 = int(m['As_Ka'].A.value) * 1.03
k1 = 0.01876
i2 = int(m['Ga_Ka'].A.value) * 1.0
k2 = 0.03083

In [ ]:
print('As Ka raw I: ', i1, '     As Ka corrected: ', i1 / k1)
print('Ga Ka raw I: ', i2, '     Ga Ka corrected: ', i2 / k2)

t1 = i1 / k1
t2 = i2 / k2
print('As:' , t1 / (t1 + t2))
print('Ga:' , t2 / (t1 + t2))

# without AZtec intensity correction
# As: 0.4943
# Ga: 0.5057

# treated as EDS_TEM


In [ ]:
files = df['File'].unique()[:-2]
files

In [ ]:
s = hs.load(path + files[4] + '.emsa', signal_type='EDS_TEM')
elements = ['As','Ga']
elements = ['Sb','Ga']
s.plot()

In [ ]:

s = hs.load(path + files[0] + '.emsa', signal_type='EDS_TEM')
elements = ['As','Ga']
s = s.isig[0.4:] # 0.4 keV

# if Vacc is lower than 20, slice at Vacc
if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
    s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]

s.add_elements(elements)
s.add_lines()

m = s.create_model()
m.fit()
m.fit_background()

In [ ]:
kfactors = [4.191,3.268] # As, Ga

intensities = s.get_lines_intensity()
m_intensities = m.get_lines_intensity()

In [ ]:
intensities

In [ ]:
m_intensities[::2]

In [ ]:
m_intensities[0].metadata.Sample.xray_lines

In [ ]:
at = s.quantification(intensities, factors=kfactors, method='CL', absorption_correction=True, thickness=10000)
at_m = s.quantification(m_intensities[::2], factors=kfactors, method='CL', absorption_correction=True, thickness=10000)

In [ ]:
print('As at%: ', at[0][0].data, 'or with model m: ', at_m[0][0].data)
print('Ga at%: ', at[0][1].data, 'or with model m: ', at_m[0][1].data)